##### Copyright 2018 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [0]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# Prática 01
A intenção desse notebook é mostrar um pouco sobre a etapa de treinamento, focando em alguns pontos sobre _overfitting_ e _underfitting_. É uma versão enxuta do notebook [overfit and underfit](https://www.tensorflow.org/tutorials/keras/overfit_and_underfit) com [Keras](https://www.tensorflow.org/guide/keras).

# "Introdução"
Nessa primeira parte iremos apenas importar os pacotes que utilizaremos posteriormente e conhecer o dataset.

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

## Download do dataset do IMDB

Em vez de usar uma incorporação como no bloco de anotações anterior, aqui codificaremos as frases com vários caracteres quentes. Esse modelo superajustará rapidamente o conjunto de treinamento. Ele será usado para demonstrar quando o ajuste excessivo ocorre e como combatê-lo.

A codificação múltipla de nossas listas significa transformá-las em vetores de 0s e 1s. Concretamente, isso significaria, por exemplo, transformar a sequência `[3, 5]` em um vetor de 10.000 dimensões que seria todo-zero, exceto os índices 3 e 5, que seriam os mesmos.

In [0]:
NUM_WORDS = 10000

(train_data, train_labels), (test_data, test_labels) = keras.datasets.imdb.load_data(num_words=NUM_WORDS)

def multi_hot_sequences(sequences, dimension):
    # Cria a matriz de zeros de dimensões (len(sequences), dimension)
    results = np.zeros((len(sequences), dimension))
    for i, word_indices in enumerate(sequences):
        results[i, word_indices] = 1.0  # atribuindo 1 a indices especificos
    return results


train_data = multi_hot_sequences(train_data, dimension=NUM_WORDS)
test_data = multi_hot_sequences(test_data, dimension=NUM_WORDS)

Discutir técnica em sala: [**One Hot Encoding**](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f).

Vejamos um dos vetores multi-quentes resultantes. Os índices de palavras são classificados por frequência, portanto, espera-se que haja mais valores 1 próximos ao índice zero, como podemos ver neste gráfico:

In [0]:
plt.plot(train_data[0])

## Demonstrar overfitting

No aprendizado profundo, o número de parâmetros aprendíveis em um modelo é frequentemente chamado de "capacidade" do modelo.

**Debate**:
- O que ocorre se a capacidade de memorização do modelo for grande?
  - E o que ocorre se essa capacidade for pequena?

Iremos agora criar alguns modelos simples para podermos mostrar a ocorrência de _overfitting_.

### Baseline model

O que estamos usando:

- [`keras.Sequential`](https://keras.io/models/sequential/): permite inserir camadas de uma rede neural em série, onde o output da primeira camada serve como input da segunda, e assim por diante.
  - [`keras.Sequential.compile()`](https://keras.io/models/sequential/#compile): configura o modelo para treinamento.
    - `optimizer`: o otimizador do modelo.
    - `loss`: função de perda.
    - `metrics`: lista de métricas a serem avaliadas pelo modelo durante o treinamento e o teste.
- [`keras.layers.Dense`](https://keras.io/layers/core/#dense): uma camada densa representa uma multiplicação de vetores de matriz.
  - `units`: quantidade de neurônios na camada.
  - `activation`: função de ativação da camada.
  - `input_shape`: dimensão da entrada.

In [0]:
# Modelo
baseline_model = keras.Sequential([
    keras.layers.Dense(16, activation='relu', input_shape=(NUM_WORDS,)),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
# Configuramos o modelo para treinamento
baseline_model.compile(optimizer='adam',
                       loss='binary_crossentropy',
                       metrics=['accuracy', 'binary_crossentropy'])
# Para visualizarmos o modelo
baseline_model.summary()

O que estamos usando:

- [`keras.model.fit()`](https://keras.io/models/model/#fit): treina o modelo em um número fixo de épocas.
  - `x`: dados inseridos.
  - `y`: dados que queremos prever.
  - `epochs`: número de épocas para treinar o modelo.
  - `batch_size`: número de amostras por atualização de gradiente.
  - `validation_data`: dados nos quais avaliar a perda e quaisquer métricas de modelo no final de cada época.
  - `verbose`: modo verboso.

In [0]:
baseline_history = baseline_model.fit(train_data,
                                      train_labels,
                                      epochs=20,
                                      batch_size=512,
                                      validation_data=(test_data, test_labels),
                                      verbose=2)

### Smaller model

Vamos criar um modelo menor que o baseline model:

In [0]:
# Modelo
smaller_model = keras.Sequential([
    keras.layers.Dense(4, activation='relu', input_shape=(NUM_WORDS,)),
    keras.layers.Dense(4, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
# Configuramos o modelo para treinamento
smaller_model.compile(optimizer='adam',
                      loss='binary_crossentropy',
                      metrics=['accuracy', 'binary_crossentropy'])
# Para visualizarmos o modelo
smaller_model.summary()

Iremos treinar o modelo usando os mesmos parâmetros do anterior.

In [0]:
smaller_history = smaller_model.fit(train_data,
                                    train_labels,
                                    epochs=20,
                                    batch_size=512,
                                    validation_data=(test_data, test_labels),
                                    verbose=2)

### Bigger model

Agora iremos criar um modelo muito maior.

In [0]:
# Modelo
bigger_model = keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(NUM_WORDS,)),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
# Configuramos o modelo para treinamento
bigger_model.compile(optimizer='adam',
                     loss='binary_crossentropy',
                     metrics=['accuracy','binary_crossentropy'])
# Para visualizarmos o modelo
bigger_model.summary()

Iremos treinar o modelo usando os mesmos parâmetros dos anteriores.

In [0]:
bigger_history = bigger_model.fit(train_data, train_labels,
                                  epochs=20,
                                  batch_size=512,
                                  validation_data=(test_data, test_labels),
                                  verbose=2)

### Visualizar o treinamento e a validação

- As linhas sólidas mostram a perda de treinamento.
- As linhas tracejadas mostram a perda de validação (lembre-se: uma menor perda de validação indica um modelo melhor).

In [0]:
def plot_history(histories, key='binary_crossentropy'):
  plt.figure(figsize=(16,10))

  for name, history in histories:
    val = plt.plot(history.epoch, history.history['val_'+key],
                   '--', label=name.title()+' Val')
    plt.plot(history.epoch, history.history[key], color=val[0].get_color(),
             label=name.title()+' Train')

  plt.xlabel('Epochs')
  plt.ylabel(key.replace('_',' ').title())
  plt.legend()

  plt.xlim([0,max(history.epoch)])


plot_history([('baseline', baseline_history),
              ('smaller', smaller_history),
              ('bigger', bigger_history)])

**Discussão**:
- Qual o melhor modelo com capacidade de generalização?

## Estratégias para evitar o overfitting

### Adicionar regularização de peso

A intenção aqui é tornarmos o nosso modelo mais simples, de forma que a entropia dos dados diminua, como também ele possa focar nos detalhes importantes das caracteristicas que ele esta aprendendo.

Para isso, iremos restringir a complexidade da rede, forçando os pesos apenas a aceitar valores pequenos, o que torna a distribuição dos valores de peso mais "regular". Aqui podemos usar duas regularizações:

- [Regularização de L1](https://developers.google.com/machine-learning/glossary/#L1_regularization), onde o custo adicionado é proporcional ao valor absoluto dos coeficientes de pesos (ou seja, ao que é chamado de "norma L1" dos pesos).

- [Regularização de L2](https://developers.google.com/machine-learning/glossary/#L2_regularization), onde o custo adicionado é proporcional ao quadrado do valor dos coeficientes de pesos (ou seja, ao que é chamado de "norma L2 quadrática" dos pesos). A regularização de L2 também é chamada de redução de peso no contexto de redes neurais. Não deixe que o nome diferente o confunda: a redução de peso é matematicamente a mesma que a regularização de L2.

A regularização L1 introduz escassez para zerar alguns dos parâmetros de peso. A regularização de L2 penalizará os parâmetros de pesos sem torná-los escassos - uma razão pela qual L2 é mais comum.

O que estamos usando para aplicar a regularização:
- `kernel_regularizer`: função reguladora aplicada à matriz de pesos.

In [0]:
# Modelo
l2_model = keras.models.Sequential([
    keras.layers.Dense(16, kernel_regularizer=keras.regularizers.l2(0.001),
                       activation='relu', input_shape=(NUM_WORDS,)),
    keras.layers.Dense(16, kernel_regularizer=keras.regularizers.l2(0.001),
                       activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
# Configuramos o modelo para treinamento
l2_model.compile(optimizer='adam',
                 loss='binary_crossentropy',
                 metrics=['accuracy', 'binary_crossentropy'])
# Treino
l2_model_history = l2_model.fit(train_data, train_labels,
                                epochs=20,
                                batch_size=512,
                                validation_data=(test_data, test_labels),
                                verbose=2)

In [0]:
plot_history([('baseline', baseline_history),
              ('l2', l2_model_history)])

Como você pode ver, o modelo regularizado L2 tornou-se muito mais resistente à adaptação excessiva do que o modelo de linha de base, embora ambos os modelos tenham o mesmo número de parâmetros.

### Adicionar dropout

O abandono é uma das técnicas de regularização mais eficazes e mais usadas para redes neurais, desenvolvida por Hinton e seus alunos na Universidade de Toronto. A desistência, aplicada a uma camada, consiste em "desistir" aleatoriamente (ou seja, zerar) uma série de recursos de saída da camada durante o treinamento. Digamos que uma determinada camada retornaria normalmente um vetor [0,2; 0,5; 1,3; 0,8; 1,1] para uma determinada amostra de entrada durante o treinamento; depois de aplicar o dropout, esse vetor terá algumas entradas zero distribuídas aleatoriamente, por exemplo, [0, 0,5, 1,3, 0, 1,1]. A "taxa de desistência" é a fração dos recursos que estão sendo zerados; geralmente é definido entre 0,2 e 0,5. No momento do teste, nenhuma unidade é eliminada e, em vez disso, os valores de saída da camada são reduzidos por um fator igual à taxa de abandono.

Estamos usando:
- [`keras.layers.Dropout`](https://keras.io/layers/core/#dropout): aplica _Dropout_ no _input_.

In [0]:
# Modelo
dpt_model = keras.models.Sequential([
    keras.layers.Dense(16, activation='relu', input_shape=(NUM_WORDS,)),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation='sigmoid')
])
# Configuramos o modelo para treinamento
dpt_model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy','binary_crossentropy'])
# Treino
dpt_model_history = dpt_model.fit(train_data, train_labels,
                                  epochs=20,
                                  batch_size=512,
                                  validation_data=(test_data, test_labels),
                                  verbose=2)

In [0]:
plot_history([('baseline', baseline_history),
              ('dropout', dpt_model_history)])

A adição de desistências é uma clara melhoria em relação ao modelo de linha de base.